To create a simple Python API using Flask and secure it with JWT (JSON Web Token) authentication, you can use the `Flask-JWT-Extended` library. This library simplifies the process of working with JWTs, making it easy to secure your API endpoints.

Here's a step-by-step guide on how to create a basic Flask API with JWT authentication:

### 1. Install Required Libraries

```bash
pip install Flask Flask-JWT-Extended
```

### 2. Set Up Flask API with JWT Authentication

Create a file named `app.py` and add the following code:

```python
from flask import Flask, jsonify, request
from flask_jwt_extended import JWTManager, create_access_token, jwt_required, get_jwt_identity

app = Flask(__name__)

# Set a secret key for JWT
app.config['JWT_SECRET_KEY'] = 'your_jwt_secret_key'  # Change this to a strong secret in production

# Initialize JWT manager
jwt = JWTManager(app)

# In-memory "database" for users
users = {
    "testuser": "testpassword"  # Example user
}

@app.route('/login', methods=['POST'])
def login():
    # Get JSON data from request
    username = request.json.get("username", None)
    password = request.json.get("password", None)

    # Check if user exists and password matches
    if users.get(username) == password:
        # Create JWT token
        access_token = create_access_token(identity=username)
        return jsonify(access_token=access_token), 200
    else:
        return jsonify({"msg": "Invalid username or password"}), 401

@app.route('/protected', methods=['GET'])
@jwt_required()  # Protect this route with JWT
def protected():
    # Access the identity of the current user with get_jwt_identity
    current_user = get_jwt_identity()
    return jsonify(logged_in_as=current_user), 200

@app.route('/public', methods=['GET'])
def public():
    # A public endpoint that doesn't require authentication
    return jsonify(message="This is a public endpoint."), 200

if __name__ == '__main__':
    app.run(debug=True)
```

### 3. Explanation of Code

1. **JWT Secret Key**: Configure a secret key (`JWT_SECRET_KEY`) to encode and decode the JWT tokens. This should be secure and kept confidential.
2. **User Data**: For simplicity, we use an in-memory dictionary (`users`) to store a sample user. In a real app, you'd query a database.
3. **Login Endpoint (`/login`)**:
   - Receives JSON data containing `username` and `password`.
   - If the credentials are valid, it generates an access token using `create_access_token`.
   - Returns the token as JSON if successful or an error message otherwise.
4. **Protected Endpoint (`/protected`)**:
   - Protected with the `@jwt_required()` decorator, meaning a valid JWT token is needed to access it.
   - Retrieves the current user's identity (`username`) from the token and returns it.
5. **Public Endpoint (`/public`)**:
   - Open to everyone, no token required.

### 4. Testing the API

1. **Start the API**: Run the Flask app:

   ```bash
   python app.py
   ```

2. **Login**: Use `curl` or Postman to make a POST request to `/login`:

   ```bash
   curl -X POST http://127.0.0.1:5000/login -H "Content-Type: application/json" -d '{"username": "testuser", "password": "testpassword"}'
   ```

   If the credentials are correct, it will return an access token.

3. **Access Protected Endpoint**: Use the token to access the `/protected` endpoint:

   ```bash
   curl -X GET http://127.0.0.1:5000/protected -H "Authorization: Bearer <your_token_here>"
   ```

This basic setup provides secure JWT token-based authentication for your API. For production, remember to use HTTPS and a strong secret key.